In [7]:
import random
import utils
import numpy as np
import matplotlib.pyplot as plt
from Game.LeducGame import *
from Environment.Environment import *
from Opponent_agent.RandAgent import *
from Opponent_agent.GreedyAgent import *
from Opponent_agent.HumanAgent import *
from Qagent.QAgent import *
from numpy import asarray
from numpy import savetxt

stack_size=5
random_agent_qtable=None
greedy_agent_qtable=None

# Training and Testing parameters

In [8]:
epochs_number=500000
evaluate_every=1
test_number=10

## qAgent vs GreedyAgent

In [9]:

greedy=GreedyAgent()
env=Environment(greedy)
envTest=Environment(greedy)
qagent=QAgent()


for i in range(epochs_number):  
    
    #TEST
    if(i % evaluate_every == 0):
        perf=0
        for j in range(test_number):
            envTest.reset()
            while(envTest.game.epoch_is_over==0):
                current_player=envTest.game.get_firstplayer()
                stack1=envTest.game.stack1
                stack2=envTest.game.stack2
                if(envTest.game.stack1<= envTest.game.stack2):
                    small_stack=envTest.game.stack1
                else:
                    small_stack=envTest.game.stack2


                allowed_actions=[0,1,2]
                if(current_player==1):
                    envTest.agent.set_action(allowed_actions,None,envTest.game.get_game_round(), envTest.game.get_hand_player2(),envTest.game.get_boardcard())
                    envTest.set_opponent_action(envTest.agent.get_action())
                    reward,current_player,allowed_actions=envTest.game.step_prime(envTest.agent.get_action(), small_stack)


                while(envTest.game.is_game_over()==0):
                    state=envTest.get_state()
                    qagent.set_state(state)
                    qagent_action=qagent.exploit_action(allowed_actions,envTest.game.stack1)

                    reward,allowed_actions, new_state=envTest.step(qagent_action, small_stack)

                    if(envTest.game.is_game_over()==0):
                        envTest.reset_card()
                        state=envTest.get_state()
                        qagent.set_state(state) 
                    if(envTest.game.epoch_is_over==1):
                        if(envTest.game.stack2==0):
                            reward=5
                        else:
                            reward=-5
                        perf=perf+reward
   
        qagent.set_perf(float(perf)/test_number)                
        envTest.reset_card()
        state=envTest.get_state()
        qagent.set_state(state)
    
    #TRAIN
    while(env.game.epoch_is_over()==0):
        
        current_player=env.game.get_firstplayer()
        stack1=env.game.stack1
        stack2=env.game.stack2

        if(env.game.stack1<= env.game.stack2):
            small_stack=env.game.stack1
        else:
            small_stack=env.game.stack2
        

        allowed_actions=[0,1,2]
        if(current_player==1):
            env.agent.set_action(allowed_actions,None,env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action(), small_stack)
            
        while(env.game.is_game_over()==0):

            state=env.get_state()
            qagent.set_state(state)
            qagent_action=qagent.explore_action(allowed_actions)

            reward,allowed_actions, new_state=env.step(qagent_action, small_stack)
            

            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist(),stack1,stack2)
                
        env.reset_card()
        state=env.get_state()
        qagent.set_state(state)

    env.reset()
    state=env.get_state()
    qagent.set_state(state)

with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)
greedy_agent_qtable=qagent.qtable

with open('test.npy', 'wb') as f:
    np.save(f, greedy_agent_qtable)
with open('test.npy', 'rb') as f:
    a = np.load(f)
print(a)

KeyboardInterrupt: 

In [ ]:

greedy=GreedyAgent()
env=Environment(greedy)
envTest=Environment(greedy)
qagent=QAgent()


for i in range(epochs_number):      
    #TRAIN
    while(env.game.epoch_is_over==0):
        
        current_player=env.game.get_firstplayer()
        stack1=env.game.stack1
        stack2=env.game.stack2

        if(env.game.stack1<= env.game.stack2):
            small_stack=env.game.stack1
        else:
            small_stack=env.game.stack2
        

        allowed_actions=[0,1,2]
        if(current_player==1):
            env.agent.set_action(allowed_actions,None,env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action(), small_stack)
            
        while(env.game.is_game_over()==0):

            state=env.get_state()
            qagent.set_state(state)
            qagent_action=qagent.explore_action(allowed_actions)

            reward,allowed_actions, new_state=env.step(qagent_action, small_stack)
            

            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist(),stack1,stack2)
                
        env.reset_card()
        state=env.get_state()
        qagent.set_state(state)

    env.reset()
    state=env.get_state()
    qagent.set_state(state)

with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)
greedy_agent_qtable=qagent.qtable

with open('big.npy', 'wb') as f:
    np.save(f, greedy_agent_qtable)
with open('big.npy', 'rb') as f:
    a = np.load(f)
print(a)

In [ ]:
import pandas as pd
import seaborn as sns
df = pd.DataFrame(qagent.qtable[8], columns=['fold', "check","push"])

cm = sns.light_palette('blue', as_cmap=True)

s = df.style.background_gradient(cmap=cm, low=0, high=1, axis=1)
s

## qAgent (trained with a greedy agent) vs HumanAgent

In [6]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()

with open('big.npy', 'rb') as f:
    a = np.load(f)
qagent.set_qtable(a)
print("How to play?\n\nFor the actions:\n\t- 0 : Fold\n\t- 1 : Check\n\t- 2 : Push\n\nFor the cards:\n\t- 0 : Jack\n\t- 1 : Queen\n\t- 2 : King\n\n ")
while(env.game.epoch_is_over==0):
    #env.display()
    print("\n==NEW GAME==")
    current_player=env.game.get_firstplayer()
    stack1=env.game.stack1
    stack2=env.game.stack2
    #print("stack1: ", stack1)
    #print("stack2: ", stack2)
    if(env.game.stack1<= env.game.stack2):
        small_stack=env.game.stack1
    else:
        small_stack=env.game.stack2
    #print("small_stack: ", small_stack)    


    allowed_actions=[0,1,2]

    #IF HUMAN AGENT
    if(current_player==1):
        print("Your stack = ",env.game.stack2)
        print("QAgent stack = ",env.game.stack1)
        print("Your card = ", env.game.hand_player2)
        env.agent.set_action(allowed_actions,None,env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
        env.set_opponent_action(env.agent.get_action())
        reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action(), small_stack)
        if(env.game.is_game_over()==1):
            print("You loose!! He has a:", env.game.get_hand_player1(), "\n\n")

        #print("opponent action: ", env.agent.get_action())

    while(env.game.is_game_over()==0):
        #print("\n NEW ")
        #print("qagent card: ", env.game.hand_player1)
        #print("board action: ", env.game.boardcard)
        #print("round : ", env.game.game_round)
        state=env.get_state()
        qagent.set_state(state)
        #print("State= ",state)
        qagent_action=qagent.exploit_action(allowed_actions, env.game.stack1)
        #print("qagent action: ", qagent_action)

        reward,allowed_actions, new_state=env.step_human(qagent_action, small_stack)


        if(env.game.is_game_over()==1):
            if(reward<0):
                print("He has a: ",env.game.get_hand_player1(),"\nThe boardcard is: ",env.game.boardcard, "\nCongratulation, you win !!!\n\n" )
            elif(reward>0):
                print("He has a: ",env.game.get_hand_player1(),"\nThe boardcard is: ",env.game.boardcard, "\nSorry, you loose !!!\n\n" )
            elif(reward==0):
                print("He has a: ",env.game.get_hand_player1(),"\nThe boardcard is: ",env.game.boardcard, "\nIt's a draw !!!\n\n" )
                
    env.reset_card()
    state=env.get_state()
    qagent.set_state(state)

if(reward<0):
    print("Well done you win all the stack of your opponent!!")
elif(reward>0):
    print("Aie, you loose all of your stack!!")
          
env.reset()
state=env.get_state()
qagent.set_state(state)

How to play?

For the actions:
	- 0 : Fold
	- 1 : Check
	- 2 : Push

For the cards:
	- 0 : Jack
	- 1 : Queen
	- 2 : King

 

==NEW GAME==
Your stack =  5
QAgent stack =  5
Your card =  2
QAgent action =  1



Please enter an action :(0,1 or 2)
2
He has a:  1 
The boardcard is:  0 
Congratulation, you win !!!



==NEW GAME==
Your stack =  6
QAgent stack =  4
Your card =  0
QAgent action =  1



Please enter an action :(0,1 or 2)
0
He has a:  0 
The boardcard is:  1 
Sorry, you loose !!!



==NEW GAME==
Your stack =  5
QAgent stack =  5
Your card =  2
QAgent action =  1



Please enter an action :(0,1 or 2)
2
He has a:  0 
The boardcard is:  1 
Congratulation, you win !!!



==NEW GAME==
Your stack =  6
QAgent stack =  4
Your card =  2
Please enter an action :(0,1 or 2)
2


He has a:  2 
The boardcard is:  1 
It's a draw !!!



==NEW GAME==
Your stack =  6
QAgent stack =  4
Your card =  2
Please enter an action :(0,1 or 2)
2


He has a:  1 
The boardcard is:  1 
Sorry, you loose !!!



=